In [22]:
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy as sc
import os
from datetime import datetime as dt

In [23]:
haf=pd.read_csv('Downloads/ForH4.csv')
db=pd.read_excel('Downloads/Book 1.xlsx')
alloc=pd.read_csv('Downloads/BL19b.csv')
coords=pd.read_csv('Downloads/bases_coords.csv')

bases_info=alloc.merge(coords).drop(['Unnamed: 3','ID'],axis=1)
bases_info['name']=bases_info['name'].apply(lambda x: x.upper().replace('FIRE POST','FP').replace('FIRE STATION','FS'))
bases_info=bases_info[bases_info['name'].apply(lambda x: ('FS' in x) or ('FP' in x))==True]
db['fb']=db['fb'].apply(lambda x:x.upper())
bases_info=bases_info.merge(db,left_on='name',right_on='fb',how='left').drop('fb',axis=1)

def find_timing(x):
    try: return dt.strptime(x,'%d/%m/%Y %I:%M:%S %p\t')
    except: return '1'

haf.iloc[:,7:15]=haf.iloc[:,7:15].applymap(find_timing)
haf['INC.CREATE.TIME']=haf['INC.CREATE.TIME'].apply(find_timing)
haf['FIRE.POST']=np.vectorize(lambda x,y: x if len(str(y)) < 5 else y+' FP')(haf['STATION'],haf['FIRE.POST'])
haf=haf.drop(['STATION','Unnamed: 0'],axis=1)
df=haf.merge(bases_info,left_on='FIRE.POST',right_on='name')
df=df.rename(columns={'lat_x':'blat','lng':'blong','lat_y':'flat','long':'flong'})

In [25]:
df['dist'] = ((df['INC.LOC.LAT']-df['flat'])**2+(df['INC.LOC.LONG']-df['flong'])**2)**0.5*111.33

In [26]:
df[df['dist']>10]

,Month,INC.CREATE.TIME,FIRE.POST,INC.LOC.LAT,INC.LOC.LONG,DISPATCH.TIME,RESPONDING.TIME,ARRIVE.SCENE.TIME,ARRIVE.PATIENT.TIME,ENROUTE.HOSPITAL,...,name,label,ambulance_count,blat,blong,db,callsign,flat,flong,dist
650,July,2019-07-10 16:33:26,CLEMENTI FS,1.343876,103.641470,2019-07-10 16:34:47,2019-07-10 16:35:21,2019-07-10 16:46:02,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,11.270980
814,July,2019-07-16 13:15:14,CLEMENTI FS,1.261298,103.664659,2019-07-16 13:18:14,2019-07-16 13:19:05,2019-07-16 13:34:37,2019-07-16 13:35:36,2019-07-16 13:50:59,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,11.590352
972,July,2019-07-22 07:27:30,CLEMENTI FS,1.312433,103.860907,2019-07-22 07:34:29,1,1,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,13.411048
973,July,2019-07-22 07:27:30,CLEMENTI FS,1.312433,103.860907,2019-07-22 07:35:30,2019-07-22 07:35:37,2019-07-22 07:43:40,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,13.411048
978,July,2019-07-22 09:52:28,CLEMENTI FS,1.336578,103.645733,2019-07-22 09:54:12,2019-07-22 09:54:48,2019-07-22 10:07:22,2019-07-22 10:10:33,2019-07-22 10:36:01,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,10.723628
1193,July,2019-07-31 16:32:28,CLEMENTI FS,1.339129,103.646794,2019-07-31 16:35:06,2019-07-31 16:35:39,2019-07-31 16:45:14,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,10.625815
1253,August,2019-08-03 06:27:52,CLEMENTI FS,1.268321,103.828233,2019-08-03 06:33:51,2019-08-03 06:34:30,2019-08-03 06:42:56,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,11.883970
1551,August,2019-08-13 16:37:50,CLEMENTI FS,1.336578,103.645733,2019-08-13 16:40:21,2019-08-13 16:41:00,2019-08-13 16:50:18,2019-08-13 16:56:29,2019-08-13 17:07:47,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,10.723628
1794,August,2019-08-22 21:10:55,CLEMENTI FS,1.313492,103.630290,2019-08-22 21:14:10,1,1,1,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,12.579223
2388,September,2019-09-12 21:47:47,CLEMENTI FS,1.347956,103.636935,2019-09-12 21:49:36,2019-09-12 21:50:21,2019-09-12 22:02:55,2019-09-12 22:08:11,1,...,CLEMENTI FS,13A,2,1.322008,103.761431,JTC Corporation,A131,1.331100,103.741900,11.835471
